In [3]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate


In [4]:
def load_pdf(data):
    loader = DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents


In [5]:
extracted_data=load_pdf('D:/abhi/programming/AI_chatbot/data')

In [6]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500,chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print(len(text_chunks))

9874


In [8]:
def hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings= hugging_face_embedding()

d:\abhi\programming\AI_chatbot\venv\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
d:\abhi\programming\AI_chatbot\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
import faiss
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings


In [11]:
def initialize_faiss(embeddings, text_chunks):
    # Create a FAISS index and add the document vectors
    faiss_store = FAISS.from_documents(text_chunks, embeddings)
    return faiss_store

# Assuming you've already split your documents into chunks
# text_chunks = text_split(extracted_data)

# Initialize the FAISS vector store
vector_store = initialize_faiss(embeddings, text_chunks)

In [12]:
vector_store.save_local("faiss_index")




In [13]:
# Load the FAISS index with the flag set to allow deserialization
faiss_store = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)


In [15]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])




In [ ]:
#this is giving incomplete sentence as output
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
from langchain.vectorstores import FAISS
import numpy as np

# Load the GPT-Neo model and tokenizer
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

def generate_response(prompt: str) -> str:
    # Tokenize the input prompt
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    
    # Generate the text with a larger max_length or max_new_tokens
    gen_tokens = model.generate(
        input_ids,
        do_sample=True,
        temperature=0.9,
        max_length=550,  # Increase this to accommodate longer prompts
        # Alternatively, use max_new_tokens to limit the length of the generated text
        max_new_tokens=100,
    )
    
    # Decode the generated tokens
    gen_text = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0]
    return gen_text

def main(query: str, faiss_store: FAISS):
    # Perform the search
    docs = faiss_store.similarity_search(query, k=5)

    # Extract the relevant context from the documents
    context = " ".join([doc.page_content for doc in docs])

    # Construct the prompt
    prompt = f"Context: {context}\nQuestion: {query}\n\nProvide a concise and accurate answer based on the context above."

    # Generate the response
    response = generate_response(prompt)
    print("Response: ", response)

# Example query
user_query = "What is the purpose of lungs?"
main(user_query, faiss_store)







In [22]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
from langchain.vectorstores import FAISS
import numpy as np

# Load the GPT-Neo model and tokenizer
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

def generate_response(prompt: str) -> str:
    gen_tokens = model.generate(
        tokenizer(prompt, return_tensors="pt").input_ids,
        do_sample=True,
        temperature=0.7,
        top_p=0.85,
        max_length=512,
        max_new_tokens=200,
    )
    gen_text = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0]
    
    # Basic post-processing
    if not gen_text.endswith(('.', '!', '?')):
        gen_text += '...'
    
    return gen_text

def main(query: str, faiss_store: FAISS):
    # Perform the search
    docs = faiss_store.similarity_search(query, k=5)

    # Extract the relevant context from the documents
    context = " ".join([doc.page_content for doc in docs])

    # Construct the prompt
    prompt = f"Context: {context}\nQuestion: {query}\n\nProvide a concise and accurate answer based on the context above."

    # Generate the response
    response = generate_response(prompt)
    print("Response: ", response)

# Example query
user_query = "What is the purpose of lungs?"
main(user_query, faiss_store)




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=200) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Response:  Context: •Outline the anatomy of the blood supply to the lungs
•Describe the pleura of the lungs and their function
A major organ of the respiratory system, each lung houses structures of both the conducting and respiratory zones. The
main function of the lungs is to perform the exchange of oxygen and carbon dioxide with air from the atmosphere. To this
end, the lungs exchange respiratory gases across a very large epithelial surface area—about 70 square meters—that is highly
permeable to gases. •Describe the process of internal respiration
The purpose of the respiratory system is to perform gas exchange. Pulmonary ventilation provides air to the alveoli for this
gas exchange process. At the respiratory membrane, where the alveolar and capillary walls meet, gases move across the
membranes, with oxygen entering the bloodstream and carbon dioxide exiting. It is through this mechanism that blood is circulation is very important, as blood is required to transport oxygen from the 